## Booze R Us Model

Fitting a model (or two) based on our proposal.

- **Goal:** Build a model to predict sales in a month for any given store.
- **Response Variable:** Monthly Sales
- **Possible Features:** store, month, county, population stuff, proximity stuff, alcohol categories

In [1]:
import duckdb as db 
con = db.connect()
import pandas as pd 
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# MAIN TABLE
con.execute("""
        DROP TABLE IF EXISTS sales;
        CREATE TABLE sales AS 
        SELECT EXTRACT(MONTH FROM date) AS month, EXTRACT (YEAR FROM date) AS year,
            store, city, county, 
            category_name AS category, sale_bottles AS bottles, sale_dollars AS dollars
        FROM read_parquet('../data/iowa_liquor_2023_2025.parquet');
""")
sales = con.execute("SELECT * FROM sales").df()

# POPULATION
con.execute(
"""
        DROP TABLE IF EXISTS pop;
        CREATE TABLE population AS
        SELECT name AS county, year_1 AS year, popestimate AS population, over21, propOver21, median_age_tot AS median_age
        FROM read_csv_auto('../data/pop.csv');
"""
)
pop = con.execute("SELECT * FROM population").df()

# PROXIMITY
con.execute(
"""
        DROP TABLE IF EXISTS prox;
        CREATE TABLE proximity AS
        SELECT *
        FROM read_csv_auto('../data/proximity.csv');
"""
)
prox = con.execute("SELECT * FROM proximity").df()

In [3]:
sales.columns

Index(['month', 'year', 'store', 'city', 'county', 'category', 'bottles',
       'dollars'],
      dtype='object')

## Creating the Dataset

First, I am going to engineer the category column a little bit to use as features. Knowing which alcohol sells the best is could be useful for telling Booze R Us what they should buy in order to increase profits.

In [4]:
con.execute("""
    CREATE OR REPLACE TABLE sales AS
    SELECT *,
        CASE
            WHEN category ILIKE '%VODKA%' THEN 'Vodka'
            WHEN category ILIKE '%WHISK%' THEN 'Whiskey'
            WHEN category ILIKE '%TEQUILA%' OR category ILIKE '%MEZCAL%' THEN 'Tequila'
            WHEN category ILIKE '%RUM%' THEN 'Rum'
            ELSE 'Other'
        END AS super_category
    FROM sales
""")
sales = con.execute("SELECT * FROM sales").df()

In [5]:
sales.head()

,month,year,store,city,county,category,bottles,dollars,super_category
0,1,2023,4829,DES MOINES,POLK,100% AGAVE TEQUILA,12,261.00,Tequila
1,1,2023,4829,DES MOINES,POLK,AMERICAN VODKAS,60,418.80,Vodka
2,1,2023,4829,DES MOINES,POLK,IMPORTED FLAVORED VODKA,24,358.56,Vodka
3,1,2023,4829,DES MOINES,POLK,CREAM LIQUEURS,12,306.00,Other
4,1,2023,4829,DES MOINES,POLK,SPICED RUM,60,1124.40,Rum


Now I need to agreggate to create our appropriate observational units: monthly sales per store.

- Dollars (our response variable) will be summed. 
- Category will be made into new columns representing the distribution of category sales
    - e.g. 70% tequila, 20% vodkas, etc.
    - we will not use total bottles because this would be almost perfectly collinear 
    - answer questions like: 'what liquor should we sell more/less of?'

In [6]:
monthly_sales = con.execute(
""" 
    WITH month_totals AS (
        SELECT year, month, store, city, county,
            SUM(dollars) AS revenue
        FROM sales
        GROUP BY year, month, store, city, county
    ), category_totals AS (
        SELECT year, month, store, city, county,
            super_category,
            SUM(dollars) AS category_sales
        FROM sales
        GROUP BY year, month, store, city, county, super_category
    )
    SELECT mt.year, mt.month, mt.store, mt.city, mt.county,
        ROUND((SUM(CASE WHEN ct.super_category = 'Vodka' THEN ct.category_sales ELSE 0 END) / mt.revenue),2) AS vodka_ptc,
        ROUND((SUM(CASE WHEN ct.super_category = 'Whiskey' THEN ct.category_sales ELSE 0 END) / mt.revenue),2) AS whiskey_ptc,
        ROUND((SUM(CASE WHEN ct.super_category = 'Tequila' THEN ct.category_sales ELSE 0 END) / mt.revenue),2) AS tequila_ptc,
        ROUND((SUM(CASE WHEN ct.super_category = 'Rum' THEN ct.category_sales ELSE 0 END) / mt.revenue),2) AS rum_ptc,
        ROUND((SUM(CASE WHEN ct.super_category = 'Other' THEN ct.category_sales ELSE 0 END) / mt.revenue),2) AS other_ptc,
        mt.revenue
    FROM month_totals mt
    LEFT JOIN category_totals ct
        ON mt.year = ct.year AND mt.month = ct.month 
            AND mt.city = ct.city AND mt.county = ct.county
            AND mt.store = ct.store 
    WHERE revenue > 0
    GROUP BY mt.year, mt.month, mt.store, mt.city, mt.county, mt.revenue
"""
).fetchdf()

Create a new feature for lagged monthly sales as a predictor. Since there is so much we don't know about individual stores (marketing, promotions, customer behavior, market share, etc...), knowing how they have done in the past can be a good indicator of future performance, making this more of a time series model.

In [7]:
monthly_sales = con.execute(
    """
        -- recreate the duckdb table from the current pandas dataframe
        DROP TABLE IF EXISTS monthly_sales;
        CREATE TABLE monthly_sales AS SELECT * FROM monthly_sales;

        -- add the monthly lagged revenue (leaving out the first month for each store)
        WITH lags AS (
            SELECT *,
                LAG(revenue) OVER (PARTITION BY store ORDER BY year ASC, month ASC) as lag_revenue
            FROM monthly_sales
        )
        SELECT *, LOG(revenue) AS log_revenue, LOG(lag_revenue) AS log_lag_revenue
        FROM lags
        WHERE lag_revenue IS NOT NULL;
    """).fetchdf()

In [8]:
monthly_sales.head(3)

,year,month,store,city,county,vodka_ptc,whiskey_ptc,tequila_ptc,rum_ptc,other_ptc,revenue,lag_revenue,log_revenue,log_lag_revenue
0,2023,2,010029,DES MOINES,POLK,0.11,0.22,0.35,0.06,0.26,15404.37,11479.52,4.187644,4.059924
1,2023,3,010029,DES MOINES,POLK,0.22,0.36,0.11,0.03,0.28,15739.55,15404.37,4.196992,4.187644
2,2023,4,010029,DES MOINES,POLK,0.20,0.41,0.08,0.02,0.29,13810.11,15739.55,4.140197,4.196992


Now I will join with our other datasets, proximity and population. Using an inner join because it still leaves plenty of complete data for modelling. 

In [9]:
df = con.execute(
    """
        DROP TABLE IF EXISTS monthly_sales;
        CREATE TABLE monthly_sales AS SELECT * FROM monthly_sales;
        SELECT sales.*, 
            pop.population, pop.over21, pop.propOver21, pop.median_age,
            prox."# of stores within 5 mile radius" AS stores_within_5_miles,
            prox."Nearest other store (mi)" AS nearest_store_miles
        FROM monthly_sales sales
        JOIN pop
            ON LOWER(sales.county) = LOWER(pop.county) AND sales.year = pop.year
        JOIN prox
            ON sales.store = prox.store
        
    """
    ).fetchdf()

In [10]:
df.head(2)

,year,month,store,city,county,vodka_ptc,whiskey_ptc,tequila_ptc,rum_ptc,other_ptc,revenue,lag_revenue,log_revenue,log_lag_revenue,population,over21,propOver21,median_age,stores_within_5_miles,nearest_store_miles
0,2023,2,010029,DES MOINES,POLK,0.11,0.22,0.35,0.06,0.26,15404.37,11479.52,4.187644,4.059924,507487,385202,0.759038,36.3,122,0.2983
1,2023,3,010029,DES MOINES,POLK,0.22,0.36,0.11,0.03,0.28,15739.55,15404.37,4.196992,4.187644,507487,385202,0.759038,36.3,122,0.2983


Now encode month to use as a categorical feature:

In [11]:
months = pd.get_dummies(df.month, prefix='month', drop_first=True)
df = pd.concat([df, months], axis=1)
df.head()

,year,month,store,city,county,vodka_ptc,whiskey_ptc,tequila_ptc,rum_ptc,other_ptc,revenue,lag_revenue,log_revenue,log_lag_revenue,population,over21,propOver21,median_age,stores_within_5_miles,nearest_store_miles,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,2023,2,010029,DES MOINES,POLK,0.11,0.22,0.35,0.06,0.26,15404.37,11479.52,4.187644,4.059924,507487,385202,0.759038,36.3,122,0.2983,True,False,False,False,False,False,False,False,False,False,False
1,2023,3,010029,DES MOINES,POLK,0.22,0.36,0.11,0.03,0.28,15739.55,15404.37,4.196992,4.187644,507487,385202,0.759038,36.3,122,0.2983,False,True,False,False,False,False,False,False,False,False,False
2,2023,4,010029,DES MOINES,POLK,0.20,0.41,0.08,0.02,0.29,13810.11,15739.55,4.140197,4.196992,507487,385202,0.759038,36.3,122,0.2983,False,False,True,False,False,False,False,False,False,False,False
3,2023,5,010029,DES MOINES,POLK,0.14,0.30,0.37,0.04,0.15,21506.33,13810.11,4.332566,4.140197,507487,385202,0.759038,36.3,122,0.2983,False,False,False,True,False,False,False,False,False,False,False
4,2023,6,010029,DES MOINES,POLK,0.22,0.35,0.16,0.03,0.24,25377.98,21506.33,4.404457,4.332566,507487,385202,0.759038,36.3,122,0.2983,False,False,False,False,True,False,False,False,False,False,False


In [12]:
df = df.copy()
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df = df[df['revenue'] > 0]   
df.to_csv('../data/brs_model_data.csv', index=False)

## Linear Regression (from scratch)

First, the function to fit the model. It will take observed X and Y matrices and return the predictions and estimators.

In [13]:
def fit_lr(X, Y):
    
    X = np.column_stack([np.ones((X.shape[0], 1)), X.astype(float)]) # add column of ones
    B = np.linalg.pinv(X.T @ X) @ X.T @ Y

    return B

In [14]:
def predict_lr(X, B):
    
    X = np.column_stack([np.ones((X.shape[0], 1)), X.astype(float)]) # add column of ones
    Y_hat = X @ B

    return Y_hat

A cross validation function:

In [17]:
def lr_cross_val(data, features, response, k=5):

    # save original indices before sorting so we can map to the correct y values later
    df = data.copy()
    df['og_idx'] = df.index

    # sort for time-based sequential folds
    data_sorted = df.sort_values(['year','month']).reset_index(drop=True)

    # create month blocks based on number of folds
    periods = (data_sorted["year"].astype(str) + '-' + data_sorted["month"].astype(str).str.zfill(2)).to_numpy()
    unique_periods = np.unique(periods)
    k = max(2, min(k, len(unique_periods)))
    month_blocks = np.array_split(unique_periods, k)

    # make predictions for each fold aligned with the sorted dataframe
    preds_sorted = pd.Series(np.nan, index=data_sorted.index, dtype=float)
    fold_rows = []
    for fold in range(1, len(month_blocks)):
        
        # training data = all folds before this fold
        # testing data = this fold only
        train_months = np.concatenate(month_blocks[:fold])
        test_months  = month_blocks[fold]
        train_mask = np.isin(periods, train_months)
        test_mask  = np.isin(periods, test_months)
        train = data_sorted.loc[train_mask]
        test  = data_sorted.loc[test_mask]

        # fit with training data and predict testing data
        B = fit_lr(train[features].values, train[response].values)
        yhat = predict_lr(test[features].values, B)
        preds_sorted.loc[test.index] = yhat

        # compute per-fold metrics
        y_true = test[response].to_numpy()
        rss = np.sum((y_true - yhat)**2)
        tss = np.sum((y_true - y_true.mean())**2)
        r2  = 1 - rss/tss if tss > 0 else np.nan
        mae = np.mean(np.abs(y_true - yhat))
        rmse = np.sqrt(np.mean((y_true - yhat)**2))
        fold_rows.append({
            "fold": fold,
            "train_months": f"{train_months[0]} to {train_months[-1]}",
            "test_months":  f"{test_months[0]} to {test_months[-1]}",
            "n_train": len(train),
            "n_test": len(test),
            "R2": r2, "MAE": mae, "RMSE": rmse
        })

    # map all predictions back using the saved original index to make full df evaluations
    preds = pd.Series(index=df['og_idx'], dtype=float)
    preds.loc[data_sorted['og_idx'].values] = preds_sorted.values
    preds = preds.reindex(data.index) 

    # calculate metrics based on the full dataset
    ## excluding the first fold because those dont get predictions
    metrics_df = pd.DataFrame(fold_rows)
    valid = preds.notna()
    y_all = data.loc[valid, response].to_numpy()
    yhat_all = preds.loc[valid].to_numpy()
    rss = np.sum((y_all - yhat_all)**2)
    tss = np.sum((y_all - y_all.mean())**2)
    overall = {
        "overall_R2": 1 - rss/tss if tss > 0 else np.nan,
        "overall_MAE": float(np.mean(np.abs(y_all - yhat_all))),
        "overall_RMSE": float(np.sqrt(np.mean((y_all - yhat_all)**2))),
        "n_eval": int(valid.sum())
    }

    return preds, metrics_df, overall


Fit the model:

- *NOTE:* I first fit with all possible features, then after assessing model fit, feature importance, and multicollinearity (all below), I came back up and changed the features. So this is just the final model here, not all iterations.

In [18]:
features = ['month_5','month_6','month_12',
        'vodka_ptc', 'whiskey_ptc', 'rum_ptc',
        'propOver21', 
        'stores_within_5_miles',
        'log_lag_revenue']
response = 'log_revenue'

# fit
yhat, fold_metrics, overall = lr_cross_val(df, features, response, k=5)

 Assess model fit:

In [19]:
fold_metrics

,fold,train_months,test_months,n_train,n_test,R2,MAE,RMSE
0,1,2023-02 to 2023-06,2023-07 to 2023-11,8289,8547,0.785979,0.169865,0.230806
1,2,2023-02 to 2023-11,2023-12 to 2024-04,16836,8639,0.791129,0.163604,0.223781
2,3,2023-02 to 2024-04,2024-05 to 2024-08,25475,6991,0.786730,0.166054,0.225059
3,4,2023-02 to 2024-08,2024-09 to 2024-12,32466,7067,0.795746,0.164545,0.222607


In [20]:
overall

{'overall_R2': 0.7898992779396427,
 'overall_MAE': 0.1660777497923604,
 'overall_RMSE': 0.22574615751059576,
 'n_eval': 31244}

Fit the finalized model now on all the data and look at the estimators:

In [21]:
# fit a final model on all the data
X = df[features]
Y = df[response]
B_final = fit_lr(X, Y)

In [22]:
coef_names = ['intercept'] + list(X.columns)
nonstd_df = pd.DataFrame({
    'feature': coef_names,
    'beta': B_final
})
nonstd_df = nonstd_df[nonstd_df['feature'] != 'intercept']
nonstd_df = nonstd_df.reindex(
    nonstd_df['beta'].abs().sort_values(ascending=False).index
)
print(nonstd_df)

                 feature      beta
9        log_lag_revenue  0.866332
4              vodka_ptc -0.183034
6                rum_ptc  0.153706
5            whiskey_ptc -0.148988
7             propOver21  0.072698
1                month_5  0.049661
2                month_6  0.006087
3               month_12  0.002143
8  stores_within_5_miles  0.000417
